In [5]:
from repositories import csvTRNSYS
import pandas as pd
from utils import transformation_EP_CO2, meanCalculation
from interactors import calculoCoste, analysisActive

In [2]:
n_zones = 1
name = 'FE35'
n_viv = 1
ref_cat_analysis = '1701521DD7810B'
typeStep = csvTRNSYS.HourlyMulti() 
output = csvTRNSYS.readTRNSYS(typeStep, n_viv, name)
output_reader, P_ht, P_cl, P_lig, P_dev, P_dhw = output.start()
n_viv_simulated = 1
building_to_analyse = 'FE35'

In [3]:
# valores standard de distribucion por hogar
#abans
#grado_ht = {'joule':0.32,'BC':0.31,'caldera_GN':0.27,'butano':0.06,'nada':0.04}
#grado_cl = {'split':0.1,'BC':0.43,'multi_split':0.04,'nada':0.43}
#grado_dhw ={'caldera_GN':0.86,'butano':0.14,'BC':0}

grado_ht = {'joule':0,'BC':1,'caldera_GN':0,'butano':0,'nada':0}
grado_cl = {'split':0,'BC':0,'multi_split':1,'nada':0}
grado_dhw ={'caldera_GN':0,'butano':0,'BC':1}
# coeficientes peninsula
coef_CO2={'ele': 0.833, 'GN': 0.252, 'butano': 0.254}
coef_NRPE={'ele': 2.937, 'GN': 1.190, 'butano': 1.201}
#EER (float) Eficiencia en refrigeración (prefinido a 2 -> CE3X)
#COP (float) -- Eficiencia en calefacción (prefinido a 2 -> CE3X)
#   COP_DHW (float) -- Eficiencia en agua caliente sanitaria con bomba de calor (prefinidos a 1.7)
#eta_cal (float) -- Eficiencia caldera gas natural o butano (prefinidos a 70% -> Optihub)
EER = 2
COP=2 
COP_DHW=1.7 
eta_cal=0.7

# càlcul d'energia primaria

PE_ht = {}
PE_cl = {}
PE_lig = {}
PE_dev = {}
PE_dhw = {}

project = transformation_EP_CO2.pE_ARV_demand(grado_ht, grado_cl, grado_dhw)
for i in P_ht:
    output = transformation_EP_CO2.calculo(project, P_ht[i], P_cl[i], P_lig[i],P_dev[i],P_dhw[i])
    PE_ht_aux, PE_cl_aux, PE_lig_aux,PE_dev_aux ,PE_dhw_aux = output.start()
    PE_ht.setdefault(i, PE_ht_aux)
    PE_cl.setdefault(i, PE_cl_aux)
    PE_lig.setdefault(i, PE_lig_aux)
    PE_dev.setdefault(i,PE_dev_aux)
    PE_dhw.setdefault(i, PE_dhw_aux)
    
# calcula horariament per un any per cada configuració de TRNSYS
pd.DataFrame(PE_ht)

,BC0_0,BC0_180,BC0_270,BC0_90,P111_0_W8_R10,P111_0_W8_R12,P111_0_W8_R14,P111_0_W8_R6,P111_0_W8_R8,P111_180_W8_R10,...,P221_90_W6_R5,P221_90_W8_R5,P222_0_W6_R5,P222_0_W8_R5,P222_180_W6_R5,P222_180_W8_R5,P222_270_W6_R5,P222_270_W8_R5,P222_90_W6_R5,P222_90_W8_R5
Dwelling_1,"[0.0, 0.021725560902761936, 0.0245716169217695...","[0.0, 0.021725560902761936, 0.0245716169217695...","[0.0, 0.021725560902761936, 0.0245716169217695...","[0.0, 0.021725560902761936, 0.0245716169217695...","[0.0, 0.02171474307260574, 0.02437654057169146...","[0.0, 0.02171474300342184, 0.02437653182694835...","[0.0, 0.021714742973846766, 0.0243765289541458...","[0.0, 0.02171474466696584, 0.02437664597921095...","[0.0, 0.021714743391190037, 0.0243765671370249...","[0.0, 0.02171474307260574, 0.02437654057169146...",...,"[0.0, 0.017406750286469888, 0.0192652747722053...","[0.0, 0.01740675028022831, 0.01926527340351150...","[0.0, 0.017406750055917755, 0.0192652642072097...","[0.0, 0.01740675004967615, 0.01926526283851584...","[0.0, 0.017406750055917755, 0.0192652642072097...","[0.0, 0.01740675004967615, 0.01926526283851584...","[0.0, 0.017406750055917755, 0.0192652642072097...","[0.0, 0.01740675004967615, 0.01926526283851584...","[0.0, 0.017406750055917755, 0.0192652642072097...","[0.0, 0.01740675004967615, 0.01926526283851584..."


In [16]:
project = analysisActive.ARV_CO2_PE(P_cl, P_ht, P_lig, P_dev, P_dhw)

In [17]:
activeAnalysis = analysisActive.do_analysis(project, grado_ht, grado_cl, grado_dhw)
PE_ht_rt, PE_cl_rt, PE_lig_rt, PE_dev_rt , PE_dhw_rt, CO2_ht_rt, CO2_cl_rt, CO2_lig_rt, CO2_dev_rt, CO2_dhw_rt = activeAnalysis.start()

In [18]:
PE_ht_rt = pd.DataFrame(PE_ht_rt)

In [19]:
PE_ht_rt = pd.concat([PE_ht_rt[col].explode().reset_index(drop=True) for col in PE_ht_rt.columns],axis=1)
PE_ht_rt

,BC0_0,BC0_180,BC0_270,BC0_90,P111_0_W8_R10,P111_0_W8_R12,P111_0_W8_R14,P111_0_W8_R6,P111_0_W8_R8,P111_180_W8_R10,...,P221_90_W6_R5,P221_90_W8_R5,P222_0_W6_R5,P222_0_W8_R5,P222_180_W6_R5,P222_180_W8_R5,P222_270_W6_R5,P222_270_W8_R5,P222_90_W6_R5,P222_90_W8_R5
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.022548,0.022548,0.022548,0.022548,0.022537,0.022537,0.022537,0.022537,0.022537,0.022537,...,0.018066,0.018066,0.018066,0.018066,0.018066,0.018066,0.018066,0.018066,0.018066,0.018066
2,0.025502,0.025502,0.025502,0.025502,0.025299,0.025299,0.025299,0.025299,0.025299,0.025299,...,0.019995,0.019995,0.019995,0.019995,0.019995,0.019995,0.019995,0.019995,0.019995,0.019995
3,0.025582,0.025582,0.025582,0.025582,0.0247,0.024699,0.024699,0.024702,0.0247,0.0247,...,0.019213,0.019213,0.019213,0.019213,0.019213,0.019213,0.019213,0.019213,0.019213,0.019213
4,0.024681,0.024681,0.024681,0.024681,0.022781,0.02278,0.022779,0.022792,0.022785,0.022781,...,0.017366,0.017366,0.017367,0.017366,0.017367,0.017366,0.017367,0.017366,0.017367,0.017366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.052877,0.052286,0.04697,0.040762,0.031853,0.031139,0.030582,0.034148,0.032805,0.031002,...,0.011098,0.010685,0.032745,0.03229,0.031926,0.03148,0.029802,0.029439,0.018504,0.018003
8756,0.052133,0.05194,0.046792,0.041255,0.03243,0.031736,0.031194,0.034666,0.033357,0.031875,...,0.013019,0.012479,0.033273,0.032826,0.032761,0.032324,0.030571,0.030225,0.020576,0.020093
8757,0.050517,0.050471,0.04572,0.040543,0.031627,0.030952,0.030426,0.033808,0.032531,0.03123,...,0.014175,0.013658,0.033014,0.032578,0.032662,0.032236,0.030522,0.030195,0.021414,0.020945
8758,0.050903,0.050895,0.046773,0.041964,0.033044,0.032409,0.031915,0.035105,0.033898,0.032749,...,0.016577,0.016095,0.033717,0.033308,0.033462,0.033061,0.031509,0.031211,0.023343,0.022905


In [25]:
pd.DataFrame(PE_dhw)

,BC0_0,BC0_180,BC0_270,BC0_90,P111_0_W8_R10,P111_0_W8_R12,P111_0_W8_R14,P111_0_W8_R6,P111_0_W8_R8,P111_180_W8_R10,...,P221_90_W6_R5,P221_90_W8_R5,P222_0_W6_R5,P222_0_W8_R5,P222_180_W6_R5,P222_180_W8_R5,P222_270_W6_R5,P222_270_W8_R5,P222_90_W6_R5,P222_90_W8_R5
Dwelling_1,"[0.0, 0.0, 0.0, 0.0, 0.0009696664850464914, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009696664850464914, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009696664850464914, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009696664850464914, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009697027193538297, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009697027490685623, 0....","[0.0, 0.0, 0.0, 0.0, 0.000969702762274592, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0009697025210500714, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009697026551173304, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009697027193538297, 0....",...,"[0.0, 0.0, 0.0, 0.0, 0.0009698066671264497, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009698066741796315, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009698066587967547, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009698066658499507, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009698066587967547, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009698066658499507, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009698066587967547, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009698066658499507, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009698066587967547, 0....","[0.0, 0.0, 0.0, 0.0, 0.0009698066658499507, 0...."


In [27]:
pd.DataFrame(PE_dhw_rt)

,BC0_0,BC0_180,BC0_270,BC0_90,P111_0_W8_R10,P111_0_W8_R12,P111_0_W8_R14,P111_0_W8_R6,P111_0_W8_R8,P111_180_W8_R10,...,P221_90_W6_R5,P221_90_W8_R5,P222_0_W6_R5,P222_0_W8_R5,P222_180_W6_R5,P222_180_W8_R5,P222_270_W6_R5,P222_270_W8_R5,P222_90_W6_R5,P222_90_W8_R5
Dwelling_1,"[0.0, 0.0, 0.0, 0.0, 0.0007377897168831999, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007377897168831999, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007377897168831999, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007377897168831999, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378172864648704, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378173090739061, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378173191219721, 0....","[0.0, 0.0, 0.0, 0.0, 0.000737817135581576, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0007378172375892731, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378172864648704, 0....",...,"[0.0, 0.0, 0.0, 0.0, 0.0007378963771614291, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378963825279805, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378963708236177, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378963761901798, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378963708236177, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378963761901798, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378963708236177, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378963761901798, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378963708236177, 0....","[0.0, 0.0, 0.0, 0.0, 0.0007378963761901798, 0...."


In [ ]:
#solo canvia el diccionario
energia_primaria_active = coef_CO2={'ele': 0.833, 'GN': 0.252, 'butano': 0.254}, coef_NRPE={'ele': 2.937, 'GN': 1.190, 'butano': 1.201}
energia_primaria_abans =coef_CO2={'ele': 0.331, 'GN': 0.252, 'butano': 0.254}  coef_NRPE={'ele': 1.954, 'GN': 1.190, 'butano': 1.201}